In [5]:
! pip install ultralytics


In [6]:
import ultralytics
ultralytics.checks()

Ultralytics YOLOv8.2.71  Python-3.9.11 torch-2.4.0+cu124 CUDA:0 (NVIDIA GeForce RTX 3050 6GB Laptop GPU, 6144MiB)
Setup complete  (12 CPUs, 15.7 GB RAM, 19.1/343.1 GB disk)


In [7]:
import cv2


In [ ]:
import cv2
import numpy as np
from ultralytics import YOLO

# Load the YOLO model
model = YOLO("K:/fine_tuned/runs/kaggle/working/runs/detect/train2/weights/best.pt")  # Replace with the path to your YOLO model

# Initialize the webcam
cap = cv2.VideoCapture(0)  # 0 for the default webcam

# Check if the webcam is opened correctly
if not cap.isOpened():
    print("Error: Could not open video stream.")
    exit()

while True:
    # Capture frame-by-frame
    ret, frame = cap.read()
    
    # Check if frame was read correctly
    if not ret:
        print("Error: Failed to capture image.")
        break

    # Convert the frame to RGB (YOLO model expects RGB images)
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    # Perform YOLO object detection
    results = model(rgb_frame)

    # Annotate the frame with bounding boxes
    for result in results:
        for box in result.boxes:
            x1, y1, x2, y2 = map(int, box.xyxy[0])
            confidence = box.conf[0]
            class_id = int(box.cls[0])
            class_name = model.names[class_id] if hasattr(model, 'names') else f"Class {class_id}"
            
            # Draw rectangle
            cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
            
            # Draw label
            label = f"{class_name} {confidence:.2f}"
            cv2.putText(frame, label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

    # Display the resulting frame
    cv2.imshow('YOLO Object Detection', frame)

    # Break the loop if 'q' is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the webcam and close windows
cap.release()
cv2.destroyAllWindows()


In [ ]:
import face_recognition

# Load the images
image1 = face_recognition.load_image_file("C:/Users/Kiran/Downloads/image1.jpg")
image2 = face_recognition.load_image_file("C:/Users/Kiran/Downloads/image3.jpg")

# Find face encodings for each image
encoding1 = face_recognition.face_encodings(image1)[0]  # Get the first face encoding in the image
encoding2 = face_recognition.face_encodings(image2)[0]  # Get the first face encoding in the image

results = face_recognition.compare_faces([encoding1], encoding2)

if results[0]:
    print("The faces match!")
else:
    print("The faces do not match.")


In [ ]:
import face_recognition
import cv2

# Load the reference image
reference_image = face_recognition.load_image_file("C:/Users/Kiran/Downloads/image2.jpg")
reference_encoding = face_recognition.face_encodings(reference_image)[0]

# Initialize the video capture
video_capture = cv2.VideoCapture(0)  # 0 for the default webcam

while True:
    # Capture frame-by-frame
    ret, frame = video_capture.read()
    if not ret:
        print("Failed to grab frame")
        break

    # Convert the frame from BGR to RGB
    rgb_frame = frame[:, :, ::-1]
    print("Frame dimensions:", rgb_frame.shape)

    # Find all face locations in the current frame
    face_locations = face_recognition.face_locations(rgb_frame)
    print("Face locations:", face_locations)


    # Check if face_locations is a list of tuples
    if not isinstance(face_locations, list):
        print("face_locations is not a list")
    
    if not all(isinstance(loc, tuple) and len(loc) == 4 for loc in face_locations):
        print("face_locations contains invalid items")

    # Find face encodings for the faces in the current frame
    face_encodings = face_recognition.face_encodings(rgb_frame, face_locations)

    # Check if face_encodings is a list
    if not isinstance(face_encodings, list):
        print("face_encodings is not a list")

    # Loop through each face found in the frame
    for (top, right, bottom, left), face_encoding in zip(face_locations, face_encodings):
        # Compare the face encoding with the reference encoding
        matches = face_recognition.compare_faces([reference_encoding], face_encoding)
        face_distance = face_recognition.face_distance([reference_encoding], face_encoding)
        
        # Check if the face matches
        if matches[0]:
            print("Face recognized!")
            # Draw a rectangle around the face
            cv2.rectangle(frame, (left, top), (right, bottom), (0, 255, 0), 2)
        else:
            print("Face not recognized.")
    
    # Display the resulting frame
    cv2.imshow('Video', frame)
    
    # Exit loop if 'q' is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the capture and close windows
video_capture.release()
cv2.destroyAllWindows()
print("Frame dimensions:", rgb_frame.shape)



In [ ]:
import cv2
import numpy as np
import face_recognition
from ultralytics import YOLO
import os

# Load the YOLO model
model = YOLO("K:/fine_tuned/runs/kaggle/working/runs/detect/train2/weights/best.pt")  # Replace with your YOLO model path

# Initialize the webcam
cap = cv2.VideoCapture(0)

# Check if the webcam is opened correctly
if not cap.isOpened():
    print("Error: Could not open video stream.")
    exit()

# Create directories for saving screenshots and logs
screenshot_dir = 'saved_screenshots'
os.makedirs(screenshot_dir, exist_ok=True)
print(f"Screenshot directory created: {os.path.abspath(screenshot_dir)}")

log_file = 'face_recognition_log.txt'
with open(log_file, 'w') as f:
    f.write("Face ID, Match Status\n")

# Initialize variables
face_encodings_dict = {}  # Dictionary to store face encodings and their IDs
recognition_threshold = 0.6  # Adjust this threshold as needed

# Global variables for tracking
tracker = None
tracking = False
selected_face_index = -1
current_face_encoding = None
current_face_id = None

# Function to save screenshot of the selected face
def save_screenshot(face_image, face_id):
    screenshot_path = os.path.join(screenshot_dir, f'face_{face_id}.png')
    cv2.imwrite(screenshot_path, face_image)
    print(f"Saved screenshot: {screenshot_path}")

# Function to log face recognition results
def log_face_recognition(face_id, is_matched):
    with open(log_file, 'a') as f:
        f.write(f"{face_id}, {1 if is_matched else 0}\n")
    print(f"Logged face ID {face_id}: {'Matched' if is_matched else 'Not Matched'}")

# Mouse callback function to select a face
def select_face(event, x, y, flags, param):
    global selected_face_index, tracker, tracking, current_face_encoding, current_face_id
    if event == cv2.EVENT_LBUTTONDOWN:
        for i, (x1, y1, x2, y2) in enumerate(face_boxes):
            if x1 <= x <= x2 and y1 <= y <= y2:
                selected_face_index = i
                # Initialize tracker with the selected bounding box
                tracker = cv2.TrackerKCF_create()
                tracker.init(frame, (x1, y1, x2 - x1, y2 - y1))
                tracking = True
                # Take screenshot of the selected face
                selected_face_image = frame[y1:y2, x1:x2]
                if selected_face_image.size > 0:
                    selected_face_image_rgb = cv2.cvtColor(selected_face_image, cv2.COLOR_BGR2RGB)
                    encodings = face_recognition.face_encodings(selected_face_image_rgb)
                    if encodings:
                        current_face_encoding = encodings[0]
                        current_face_id = len(face_encodings_dict) + 1  # Unique ID
                        face_encodings_dict[current_face_id] = current_face_encoding
                        # Save screenshot with a unique identifier
                        save_screenshot(selected_face_image, current_face_id)
                break

# Function to recognize face
def recognize_face(face_encoding):
    global face_encodings_dict
    face_distances = face_recognition.face_distance(list(face_encodings_dict.values()), face_encoding)
    if len(face_distances) > 0:
        best_match_index = np.argmin(face_distances)
        if face_distances[best_match_index] < recognition_threshold:
            return list(face_encodings_dict.keys())[best_match_index]
    return None

# Set mouse callback function
cv2.namedWindow("YOLO Object Detection")
cv2.setMouseCallback("YOLO Object Detection", select_face)

while True:
    # Capture frame-by-frame
    ret, frame = cap.read()
    
    # Check if frame was read correctly
    if not ret:
        print("Error: Failed to capture image.")
        break

    if tracking and tracker is not None:
        # Update the tracker and get the updated position
        ok, bbox = tracker.update(frame)
        if ok:
            x1, y1, w, h = map(int, bbox)
            x2, y2 = x1 + w, y1 + h
            # Draw the bounding box and label
            cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 0, 255), 2)  # Red for tracking
            cv2.putText(frame, "Tracking", (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 2)

            # Perform face recognition
            face_image = frame[y1:y2, x1:x2]
            if face_image.size > 0:
                face_image_rgb = cv2.cvtColor(face_image, cv2.COLOR_BGR2RGB)
                face_encodings = face_recognition.face_encodings(face_image_rgb)
                if face_encodings:
                    face_encoding = face_encodings[0]
                    face_id = recognize_face(face_encoding)
                    if face_id:
                        current_face_id = face_id
                        cv2.putText(frame, f"Recognized ID: {current_face_id}", (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 2)
                        log_face_recognition(current_face_id, True)
                    else:
                        # If face not recognized, stop tracking
                        tracking = False
                        tracker = None
                        current_face_encoding = None
                        current_face_id = None
                        log_face_recognition(current_face_id, False)
        else:
            # Tracker failed, so stop tracking
            tracking = False
            tracker = None
            current_face_encoding = None
            current_face_id = None
    else:
        # Convert the frame to RGB (YOLO model expects RGB images)
        rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

        # Perform YOLO object detection
        results = model(rgb_frame)

        # Reset face_boxes list
        face_boxes = []

        # Annotate the frame with bounding boxes
        for result in results:
            for box in result.boxes:
                x1, y1, x2, y2 = map(int, box.xyxy[0])
                confidence = box.conf[0]
                class_id = int(box.cls[0])
                class_name = model.names[class_id] if hasattr(model, 'names') else f"Class {class_id}"

                # Store face box coordinates
                face_boxes.append((x1, y1, x2, y2))

                # Draw rectangle
                color = (0, 255, 0)  # Green for untracked faces
                if face_boxes.index((x1, y1, x2, y2)) == selected_face_index and selected_face_index >= 0:
                    color = (0, 0, 255)  # Red for selected face
                cv2.rectangle(frame, (x1, y1), (x2, y2), color, 2)

                # Draw label
                label = f"{class_name} {confidence:.2f}"
                cv2.putText(frame, label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)

    # Display the resulting frame
    cv2.imshow('YOLO Object Detection', frame)

    # Break the loop if 'q' is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the webcam and close windows
cap.release()
cv2.destroyAllWindows()


In [1]:
import cv2
import numpy as np
import face_recognition
from ultralytics import YOLO
import os

# Load the YOLO model
model = YOLO("K:/fine_tuned/runs/kaggle/working/runs/detect/train2/weights/best.pt")  # Replace with your YOLO model path

# Initialize the webcam
cap = cv2.VideoCapture(0)

# Check if the webcam is opened correctly
if not cap.isOpened():
    print("Error: Could not open video stream.")
    exit()

# Create a directory for saving screenshots
screenshot_dir = 'saved_screenshots'
os.makedirs(screenshot_dir, exist_ok=True)
print(f"Screenshot directory created: {os.path.abspath(screenshot_dir)}")

# Initialize variables
previous_face_encodings = {}
recognition_done = False
recognition_threshold = 0.6  # Adjust this threshold as needed

# Global variables to store face boxes and selected face
face_boxes = []
selected_face_index = -1
tracker = None
tracking = False
current_face_encoding = None

# Function to save screenshot of the selected face
def save_screenshot(face_image, face_id):
    screenshot_path = os.path.join(screenshot_dir, f'face_{face_id}.png')
    cv2.imwrite(screenshot_path, face_image)
    print(f"Saved screenshot: {screenshot_path}")

# Function to compare the current face encoding with previously saved encodings
def compare_face(new_encoding, tolerance=0.6):
    for face_id, known_encoding in previous_face_encodings.items():
        matches = face_recognition.compare_faces([known_encoding], new_encoding, tolerance=tolerance)
        if matches[0]:
            return face_id
    return None

# Function to save comparison results to a log file
def save_comparison_result(face_id, match_status):
    log_path = 'face_comparison_log.txt'
    with open(log_path, 'a') as log_file:
        log_file.write(f"Face ID: {face_id}, Match Status: {'True' if match_status else 'False'}\n")
    print(f"Saved comparison result to log file: {log_path}")

# Mouse callback function to select a face
def select_face(event, x, y, flags, param):
    global selected_face_index, tracker, tracking, recognition_done, current_face_encoding
    if event == cv2.EVENT_LBUTTONDOWN:
        for i, (x1, y1, x2, y2) in enumerate(face_boxes):
            if x1 <= x <= x2 and y1 <= y <= y2:
                selected_face_index = i
                # Initialize tracker with the selected bounding box
                tracker = cv2.TrackerKCF_create()
                tracker.init(frame, (x1, y1, x2 - x1, y2 - y1))
                tracking = True
                recognition_done = False  # Reset recognition flag
                # Take screenshot of the selected face
                selected_face_image = frame[y1:y2, x1:x2]
                if selected_face_image.size > 0:
                    selected_face_image_rgb = cv2.cvtColor(selected_face_image, cv2.COLOR_BGR2RGB)
                    encodings = face_recognition.face_encodings(selected_face_image_rgb)
                    if encodings:
                        current_face_encoding = encodings[0]
                        # Save screenshot with a unique identifier
                        save_screenshot(selected_face_image, selected_face_index)
                        # Save the encoding for future comparison
                        previous_face_encodings[selected_face_index] = current_face_encoding
                        print(f"Face selected and screenshot taken: {selected_face_index}")
                else:
                    print("Warning: Selected face image size is 0.")
                break

# Set mouse callback function
cv2.namedWindow("YOLO Object Detection")
cv2.setMouseCallback("YOLO Object Detection", select_face)

while True:
    # Capture frame-by-frame
    ret, frame = cap.read()
    
    # Check if frame was read correctly
    if not ret:
        print("Error: Failed to capture image.")
        break

    if tracking and tracker is not None:
        # Update the tracker and get the updated position
        ok, bbox = tracker.update(frame)
        if ok:
            x1, y1, w, h = map(int, bbox)
            x2, y2 = x1 + w, y1 + h
            # Draw the bounding box and label
            cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 0, 255), 2)  # Red for tracking
            cv2.putText(frame, "Tracking", (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 2)

            # If face recognition hasn't been done yet
            if not recognition_done and current_face_encoding is not None:
                face_image = frame[y1:y2, x1:x2]
                if face_image.size > 0:
                    face_image_rgb = cv2.cvtColor(face_image, cv2.COLOR_BGR2RGB)
                    face_encodings = face_recognition.face_encodings(face_image_rgb)
                    if face_encodings:
                        face_encoding = face_encodings[0]
                        match_face_id = compare_face(face_encoding, tolerance=recognition_threshold)
                        match_status = match_face_id is not None
                        save_comparison_result(selected_face_index, match_status)
                        if match_status:
                            recognition_done = True
                            cv2.putText(frame, "Recognized", (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 2)
        else:
            tracking = False
            current_face_encoding = None
    else:
        # Convert the frame to RGB (YOLO model expects RGB images)
        rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

        # Perform YOLO object detection
        results = model(rgb_frame)

        # Reset face_boxes list
        face_boxes = []

        # Annotate the frame with bounding boxes
        for result in results:
            for box in result.boxes:
                x1, y1, x2, y2 = map(int, box.xyxy[0])
                confidence = box.conf[0]
                class_id = int(box.cls[0])
                class_name = model.names[class_id] if hasattr(model, 'names') else f"Class {class_id}"

                # Store face box coordinates
                face_boxes.append((x1, y1, x2, y2))

                # Draw rectangle
                color = (0, 255, 0)  # Green for untracked faces
                if face_boxes.index((x1, y1, x2, y2)) == selected_face_index and selected_face_index >= 0:
                    color = (0, 0, 255)  # Red for selected face
                cv2.rectangle(frame, (x1, y1), (x2, y2), color, 2)

                # Draw label
                label = f"{class_name} {confidence:.2f}"
                cv2.putText(frame, label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)

    # Display the resulting frame
    cv2.imshow("YOLO Object Detection", frame)

    # Break the loop on 'q' key press
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the webcam and close windows
cap.release()
cv2.destroyAllWindows()


Screenshot directory created: k:\fine_tuned\saved_screenshots

0: 480x640 1 face, 51.0ms
Speed: 3.0ms preprocess, 51.0ms inference, 76.5ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 face, 16.0ms
Speed: 2.0ms preprocess, 16.0ms inference, 4.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 face, 15.0ms
Speed: 4.0ms preprocess, 15.0ms inference, 3.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 face, 13.0ms
Speed: 2.0ms preprocess, 13.0ms inference, 2.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 face, 12.0ms
Speed: 2.0ms preprocess, 12.0ms inference, 4.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 face, 11.0ms
Speed: 1.0ms preprocess, 11.0ms inference, 3.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 face, 10.5ms
Speed: 1.0ms preprocess, 10.5ms inference, 3.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 face, 11.0ms
Speed: 1.0ms preprocess, 11.0ms inference,

In [3]:
import cv2
import numpy as np
import face_recognition
from ultralytics import YOLO
import os

# Load the YOLO model
model = YOLO("K:/fine_tuned/runs/kaggle/working/runs/detect/train2/weights/best.pt")  # Replace with your YOLO model path

# Initialize the webcam
cap = cv2.VideoCapture(0)

# Check if the webcam is opened correctly
if not cap.isOpened():
    print("Error: Could not open video stream.")
    exit()

# Load known face encodings (initially empty)
reference_face_encoding = None
recognition_threshold = 0.6  # Adjust this threshold as needed

# Global variables to store face boxes and selected face
face_boxes = []
selected_face_index = -1
tracker = None
tracking = False
recognition_done = False

# Mouse callback function to select a face
def select_face(event, x, y, flags, param):
    global selected_face_index, reference_face_encoding, recognition_done, tracker, tracking
    if event == cv2.EVENT_LBUTTONDOWN:
        for i, (x1, y1, x2, y2) in enumerate(face_boxes):
            if x1 <= x <= x2 and y1 <= y <= y2:
                selected_face_index = i

                # Take a screenshot of the selected face
                selected_face_image = frame[y1:y2, x1:x2]
                if selected_face_image.size > 0:
                    selected_face_image_rgb = cv2.cvtColor(selected_face_image, cv2.COLOR_BGR2RGB)
                    encodings = face_recognition.face_encodings(selected_face_image_rgb)
                    if encodings:
                        reference_face_encoding = encodings[0]
                        print("Screenshot taken and face encoding stored.")

                        # Initialize tracker with the selected bounding box
                        tracker = cv2.TrackerKCF_create()
                        tracker.init(frame, (x1, y1, x2 - x1, y2 - y1))
                        tracking = True
                        recognition_done = False  # Reset recognition flag
                        print("Tracking initialized for selected face.")
                break

# Function to play a sound
def play_sound(filename):
    os.system(f'start {filename}')  # Works on Windows

# Set mouse callback function
cv2.namedWindow("YOLO Object Detection")
cv2.setMouseCallback("YOLO Object Detection", select_face)

while True:
    # Capture frame-by-frame
    ret, frame = cap.read()
    
    # Check if frame was read correctly
    if not ret:
        print("Error: Failed to capture image.")
        break

    if tracking and tracker is not None:
        # Update the tracker and get the updated position
        ok, bbox = tracker.update(frame)
        if ok:
            x1, y1, w, h = map(int, bbox)
            x2, y2 = x1 + w, y1 + h
            # Draw the bounding box and label
            cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 0, 255), 2)  # Red for tracking
            cv2.putText(frame, "Tracking", (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 2)

            # If face recognition hasn't been done yet
            if not recognition_done and reference_face_encoding is not None:
                face_image = frame[y1:y2, x1:x2]
                if face_image.size > 0:
                    face_image_rgb = cv2.cvtColor(face_image, cv2.COLOR_BGR2RGB)
                    face_encodings = face_recognition.face_encodings(face_image_rgb)
                    if face_encodings:
                        face_encoding = face_encodings[0]
                        face_distances = face_recognition.face_distance([reference_face_encoding], face_encoding)
                        similarity_score = face_distances[0]
                        print(f"Face distance: {similarity_score:.2f}")  # Debugging output
                        if similarity_score < recognition_threshold:
                            recognition_done = True
                            cv2.putText(frame, "Recognized", (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 2)
                            cv2.putText(frame, f"Match Score: {similarity_score:.2f}", (x1, y2 + 20), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 2)
                            play_sound("match_sound.mp3")  # Play a sound on match
                            print("Face matched!")
                        else:
                            cv2.putText(frame, "Not Recognized", (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 255), 2)
                            cv2.putText(frame, f"Mismatch Score: {similarity_score:.2f}", (x1, y2 + 20), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 255), 2)
                            print("Face did not match.")
        else:
            # Lost tracking
            print("Lost tracking. Looking for reappearance.")
            tracking = False
            recognition_done = False

            # Convert the frame to RGB (YOLO model expects RGB images)
            rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

            # Perform YOLO object detection
            results = model(rgb_frame)

            # Reset face_boxes list
            face_boxes = []

            # Annotate the frame with bounding boxes
            for result in results:
                for box in result.boxes:
                    x1, y1, x2, y2 = map(int, box.xyxy[0])
                    confidence = box.conf[0]
                    class_id = int(box.cls[0])
                    class_name = model.names[class_id] if hasattr(model, 'names') else f"Class {class_id}"

                    # Store face box coordinates
                    face_boxes.append((x1, y1, x2, y2))

                    # Check for reappearance of the reference face
                    if reference_face_encoding is not None:
                        detected_face_image = frame[y1:y2, x1:x2]
                        if detected_face_image.size > 0:
                            detected_face_image_rgb = cv2.cvtColor(detected_face_image, cv2.COLOR_BGR2RGB)
                            encodings = face_recognition.face_encodings(detected_face_image_rgb)
                            if encodings:
                                detected_face_encoding = encodings[0]
                                face_distance = face_recognition.face_distance([reference_face_encoding], detected_face_encoding)[0]
                                print(f"Detected face distance: {face_distance:.2f}")
                                if face_distance < recognition_threshold:
                                    # Reinitialize tracking for the reappeared face
                                    tracker = cv2.TrackerKCF_create()
                                    tracker.init(frame, (x1, y1, x2 - x1, y2 - y1))
                                    tracking = True
                                    recognition_done = True
                                    selected_face_index = face_boxes.index((x1, y1, x2, y2))
                                    print("Reinitialized tracking for reappeared face.")
                                    cv2.putText(frame, "Face Reappeared", (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 2)
                                    play_sound("reappear_sound.mp3")
                                    break

                    # Draw rectangle
                    color = (0, 255, 0)  # Green for untracked faces
                    cv2.rectangle(frame, (x1, y1), (x2, y2), color, 2)

                    # Draw label
                    label = f"{class_name} {confidence:.2f}"
                    cv2.putText(frame, label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)

    else:
        # Convert the frame to RGB (YOLO model expects RGB images)
        rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

        # Perform YOLO object detection
        results = model(rgb_frame)

        # Reset face_boxes list
        face_boxes = []

        # Annotate the frame with bounding boxes
        for result in results:
            for box in result.boxes:
                x1, y1, x2, y2 = map(int, box.xyxy[0])
                confidence = box.conf[0]
                class_id = int(box.cls[0])
                class_name = model.names[class_id] if hasattr(model, 'names') else f"Class {class_id}"

                # Store face box coordinates
                face_boxes.append((x1, y1, x2, y2))

                # Check for reappearance of the reference face
                if reference_face_encoding is not None:
                    detected_face_image = frame[y1:y2, x1:x2]
                    if detected_face_image.size > 0:
                        detected_face_image_rgb = cv2.cvtColor(detected_face_image, cv2.COLOR_BGR2RGB)
                        encodings = face_recognition.face_encodings(detected_face_image_rgb)
                        if encodings:
                            detected_face_encoding = encodings[0]
                            face_distance = face_recognition.face_distance([reference_face_encoding], detected_face_encoding)[0]
                            print(f"Detected face distance: {face_distance:.2f}")
                            if face_distance < recognition_threshold:
                                # Initialize tracker for the matching face
                                tracker = cv2.TrackerKCF_create()
                                tracker.init(frame, (x1, y1, x2 - x1, y2 - y1))
                                tracking = True
                                recognition_done = True
                                selected_face_index = face_boxes.index((x1, y1, x2, y2))
                                print("Tracking initialized for matching face.")
                                cv2.putText(frame, "Face Matched", (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 2)
                                play_sound("C:/Users/Kiran/Downloads/Face detected.mp3")
                                break

                # Draw rectangle
                color = (0, 255, 0)  # Green for untracked faces
                cv2.rectangle(frame, (x1, y1), (x2, y2), color, 2)

                # Draw label
                label = f"{class_name} {confidence:.2f}"
                cv2.putText(frame, label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)

    # Display the resulting frame
    cv2.imshow('YOLO Object Detection', frame)

    # Break the loop if 'q' is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the webcam and close windows
cap.release()
cv2.destroyAllWindows()



0: 480x640 1 face, 49.1ms
Speed: 2.0ms preprocess, 49.1ms inference, 128.5ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 face, 14.0ms
Speed: 2.0ms preprocess, 14.0ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 face, 14.0ms
Speed: 1.0ms preprocess, 14.0ms inference, 2.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 face, 18.0ms
Speed: 2.0ms preprocess, 18.0ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 face, 12.0ms
Speed: 1.0ms preprocess, 12.0ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 face, 10.0ms
Speed: 1.0ms preprocess, 10.0ms inference, 3.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 face, 12.0ms
Speed: 1.0ms preprocess, 12.0ms inference, 2.5ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 face, 10.0ms
Speed: 1.0ms preprocess, 10.0ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 640)

0: 48

In [1]:
import cv2
import numpy as np
import face_recognition
from ultralytics import YOLO
import os

# Load the YOLO model
model = YOLO("K:/fine_tuned/runs/kaggle/working/runs/detect/train2/weights/best.pt")  # Replace with your YOLO model path

# Initialize the webcam
cap = cv2.VideoCapture(0)

# Check if the webcam is opened correctly
if not cap.isOpened():
    print("Error: Could not open video stream.")
    exit()

# Load known face encodings (initially empty)
reference_face_encoding = None
recognition_threshold = 0.6  # Adjust this threshold as needed

# Global variables to store face boxes and selected face
face_boxes = []
selected_face_index = -1
tracker = None
tracking = False
recognition_done = False

# Mouse callback function to select a face
def select_face(event, x, y, flags, param):
    global selected_face_index, reference_face_encoding, recognition_done, tracker, tracking
    if event == cv2.EVENT_LBUTTONDOWN:
        for i, (x1, y1, x2, y2) in enumerate(face_boxes):
            if x1 <= x <= x2 and y1 <= y <= y2:
                selected_face_index = i

                # Take a screenshot of the selected face
                selected_face_image = frame[y1:y2, x1:x2]
                if selected_face_image.size > 0:
                    selected_face_image_rgb = cv2.cvtColor(selected_face_image, cv2.COLOR_BGR2RGB)
                    encodings = face_recognition.face_encodings(selected_face_image_rgb)
                    if encodings:
                        reference_face_encoding = encodings[0]
                        print("Screenshot taken and face encoding stored.")

                        # Initialize tracker with the selected bounding box
                        tracker = cv2.TrackerKCF_create()
                        tracker.init(frame, (x1, y1, x2 - x1, y2 - y1))
                        tracking = True
                        recognition_done = False  # Reset recognition flag
                        print("Tracking initialized for selected face.")
                break

# Set mouse callback function
cv2.namedWindow("YOLO Object Detection")
cv2.setMouseCallback("YOLO Object Detection", select_face)

while True:
    # Capture frame-by-frame
    ret, frame = cap.read()
    
    # Check if frame was read correctly
    if not ret:
        print("Error: Failed to capture image.")
        break

    if tracking and tracker is not None:
        # Update the tracker and get the updated position
        ok, bbox = tracker.update(frame)
        if ok:
            x1, y1, w, h = map(int, bbox)
            x2, y2 = x1 + w, y1 + h
            # Draw the bounding box and label
            cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 0, 255), 2)  # Red for tracking
            cv2.putText(frame, "Tracking", (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 2)

            # If face recognition hasn't been done yet
            if not recognition_done and reference_face_encoding is not None:
                face_image = frame[y1:y2, x1:x2]
                if face_image.size > 0:
                    face_image_rgb = cv2.cvtColor(face_image, cv2.COLOR_BGR2RGB)
                    face_encodings = face_recognition.face_encodings(face_image_rgb)
                    if face_encodings:
                        face_encoding = face_encodings[0]
                        face_distances = face_recognition.face_distance([reference_face_encoding], face_encoding)
                        similarity_score = face_distances[0]
                        print(f"Face distance: {similarity_score:.2f}")  # Debugging output
                        if similarity_score < recognition_threshold:
                            recognition_done = True
                            cv2.putText(frame, "Recognized", (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 2)
                            cv2.putText(frame, f"Match Score: {similarity_score:.2f}", (x1, y2 + 20), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 2)
                            print("Face matched!")
                        else:
                            cv2.putText(frame, "Not Recognized", (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 255), 2)
                            cv2.putText(frame, f"Mismatch Score: {similarity_score:.2f}", (x1, y2 + 20), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 255), 2)
                            print("Face did not match.")
        else:
            # Lost tracking
            print("Lost tracking. Looking for reappearance.")
            tracking = False
            recognition_done = False

            # Convert the frame to RGB (YOLO model expects RGB images)
            rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

            # Perform YOLO object detection
            results = model(rgb_frame)

            # Reset face_boxes list
            face_boxes = []

            # Annotate the frame with bounding boxes
            for result in results:
                for box in result.boxes:
                    x1, y1, x2, y2 = map(int, box.xyxy[0])
                    confidence = box.conf[0]
                    class_id = int(box.cls[0])
                    class_name = model.names[class_id] if hasattr(model, 'names') else f"Class {class_id}"

                    # Store face box coordinates
                    face_boxes.append((x1, y1, x2, y2))

                    # Check for reappearance of the reference face
                    if reference_face_encoding is not None:
                        detected_face_image = frame[y1:y2, x1:x2]
                        if detected_face_image.size > 0:
                            detected_face_image_rgb = cv2.cvtColor(detected_face_image, cv2.COLOR_BGR2RGB)
                            encodings = face_recognition.face_encodings(detected_face_image_rgb)
                            if encodings:
                                detected_face_encoding = encodings[0]
                                face_distance = face_recognition.face_distance([reference_face_encoding], detected_face_encoding)[0]
                                print(f"Detected face distance: {face_distance:.2f}")
                                if face_distance < recognition_threshold:
                                    # Reinitialize tracking for the reappeared face
                                    tracker = cv2.TrackerKCF_create()
                                    tracker.init(frame, (x1, y1, x2 - x1, y2 - y1))
                                    tracking = True
                                    recognition_done = True
                                    selected_face_index = face_boxes.index((x1, y1, x2, y2))
                                    print("Reinitialized tracking for reappeared face.")
                                    cv2.putText(frame, "Face Reappeared", (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 2)
                                    break

                    # Draw rectangle
                    color = (0, 255, 0)  # Green for untracked faces
                    cv2.rectangle(frame, (x1, y1), (x2, y2), color, 2)

                    # Draw label
                    label = f"{class_name} {confidence:.2f}"
                    cv2.putText(frame, label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)

    else:
        # Convert the frame to RGB (YOLO model expects RGB images)
        rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

        # Perform YOLO object detection
        results = model(rgb_frame)

        # Reset face_boxes list
        face_boxes = []

        # Annotate the frame with bounding boxes
        for result in results:
            for box in result.boxes:
                x1, y1, x2, y2 = map(int, box.xyxy[0])
                confidence = box.conf[0]
                class_id = int(box.cls[0])
                class_name = model.names[class_id] if hasattr(model, 'names') else f"Class {class_id}"

                # Store face box coordinates
                face_boxes.append((x1, y1, x2, y2))

                # Check for reappearance of the reference face
                if reference_face_encoding is not None:
                    detected_face_image = frame[y1:y2, x1:x2]
                    if detected_face_image.size > 0:
                        detected_face_image_rgb = cv2.cvtColor(detected_face_image, cv2.COLOR_BGR2RGB)
                        encodings = face_recognition.face_encodings(detected_face_image_rgb)
                        if encodings:
                            detected_face_encoding = encodings[0]
                            face_distance = face_recognition.face_distance([reference_face_encoding], detected_face_encoding)[0]
                            print(f"Detected face distance: {face_distance:.2f}")
                            if face_distance < recognition_threshold:
                                # Initialize tracker for the matching face
                                tracker = cv2.TrackerKCF_create()
                                tracker.init(frame, (x1, y1, x2 - x1, y2 - y1))
                                tracking = True
                                recognition_done = True
                                selected_face_index = face_boxes.index((x1, y1, x2, y2))
                                print("Tracking initialized for detected reference face.")
                                cv2.putText(frame, "Reference Face Detected", (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 2)
                                break

                # Draw rectangle
                color = (0, 255, 0)  # Green for untracked faces
                cv2.rectangle(frame, (x1, y1), (x2, y2), color, 2)

                # Draw label
                label = f"{class_name} {confidence:.2f}"
                cv2.putText(frame, label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)

    # Display the resulting frame
    cv2.imshow("YOLO Object Detection", frame)

    # Break the loop on 'q' key press
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# When everything is done, release the capture
cap.release()
cv2.destroyAllWindows()



0: 480x640 1 face, 72.0ms
Speed: 3.0ms preprocess, 72.0ms inference, 90.3ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 face, 11.0ms
Speed: 4.0ms preprocess, 11.0ms inference, 2.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 face, 10.0ms
Speed: 1.0ms preprocess, 10.0ms inference, 2.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 face, 15.0ms
Speed: 1.5ms preprocess, 15.0ms inference, 2.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 face, 13.0ms
Speed: 2.0ms preprocess, 13.0ms inference, 2.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 face, 13.0ms
Speed: 1.0ms preprocess, 13.0ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 face, 13.0ms
Speed: 1.0ms preprocess, 13.0ms inference, 3.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 face, 12.0ms
Speed: 2.0ms preprocess, 12.0ms inference, 2.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480